<a href="https://colab.research.google.com/github/frankwillard/NBA-Hall-Of-Fame-Model/blob/main/Hall_of_Fame_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [45]:
# read in data
model_df = pd.read_csv("https://raw.githubusercontent.com/frankwillard/NBA-Hall-Of-Fame-Model/main/Scraped%20Player%20Data.csv", index_col=0)
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4977 entries, 0 to 4976
Data columns (total 67 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          4977 non-null   object 
 1   Eligible        4977 non-null   int64  
 2   Position        4977 non-null   object 
 3   Hall_of_Fame    4977 non-null   int64  
 4   MVP             4977 non-null   int64  
 5   Finals_MVP      4977 non-null   int64  
 6   NBA_Champ       4977 non-null   int64  
 7   All_NBA         4977 non-null   int64  
 8   All_Defensive   4977 non-null   int64  
 9   Def_POY         4977 non-null   int64  
 10  All_Star        4977 non-null   int64  
 11  Scoring_Champ   4977 non-null   int64  
 12  TRB_Champ       4977 non-null   int64  
 13  AST_Champ       4977 non-null   int64  
 14  STL_Champ       4977 non-null   int64  
 15  BLK_Champ       4977 non-null   int64  
 16  All_ABA         4977 non-null   int64  
 17  ABA_Champ       4977 non-null   i

In [61]:
model_df = model_df.replace(-999, np.nan)

In [62]:
# Reduce number of positions
model_df.loc[model_df['Position'] == 'Center/Forward', 'Position'] = 'Center'
model_df.loc[model_df['Position'].isin(['PointGuard', 'ShootingGuard', 'Guard/Forward']), 'Position'] = 'Guard'
model_df.loc[model_df['Position'].isin(['SmallForward', 'PowerForward', 'Forward/Guard', 'Forward/Center']), 'Position'] = 'Forward'

In [74]:
# Add ABA and NBA accolades
model_df['All_League'] = model_df['All_NBA'] + model_df['All_ABA']
model_df['Champ'] = model_df['NBA_Champ'] + model_df['ABA_Champ']

In [75]:
#Columns with NAs:
for col in model_df.columns:
  if len(model_df[model_df[col].isna()]) > 0:
    print(col, "-", len(model_df[model_df[col].isna()]))

2P_per_game - 1118
2PA_per_game - 1118
ORB_per_game - 949
DRB_per_game - 949
GS_totals - 1689
2P_totals - 1118
2PA_totals - 1118
ORB_totals - 949
DRB_totals - 949
Trp_Dbl_totals - 4526
OWS_advanced - 1
DWS_advanced - 1
WS/48_advanced - 344
OBPM_advanced - 1185
DBPM_advanced - 1185


In [76]:
# Columns with -999s
for col in model_df.columns:
  if len(model_df[model_df[col] == -999]) > 0:
    print(col, "-", len(model_df[model_df[col] == -999]))

In [77]:
# WHAT TO DO WITH NAs / -999

# Columns to drop:
# GS_totals, Trp_Dbl_totals, ORB_per_game, DRB_per_game, ORB_totals, DRB_totals, 3P%_totals, 2P%_totals, eFG%_totals, OWS_advanced, DWS_advanced, WS/48_advanced, OBPM_advanced, DBPM_advanced

# Columns to drop for now: (just for bare bones model, consider bringing back FG% FT% 2PT stuff at least)
# 3P_per_game, 3PA_per_game, 3P_totals, 3PA_totals
# FG%_totals (these players never took a shot)
# FT%_totals (these players never took a FT)
# Columns to take from FGM, FGA, etc.
# 2P_per_game, 2PA_per_game, 2P_totals, 2PA_totals

# Columns to fill with league average
# PER_advanced, VORP_advanced (consider some more advanced PER/VORP)

# Columns to make 0
# WS_advanced, BPM_advanced (consider some more advance imputation for BPM)

# Columns to make 0 or fill with mean (undecided):***
# TS%_advanced (these players never took a shot or free throw?)

# Columns to fill with mean (potentially by position):
# TRB_per_game, AST_per_game, STL_per_game, BLK_per_game, TRB_totals, AST_totals, STL_totals, BLK_totals

In [78]:
def fillNulls(model_df):
  cols_to_zero = ['WS_advanced', 'BPM_advanced', '3P_per_game', '3PA_per_game', '3P_totals', '3PA_totals', 'FG%_totals', 'FT%_totals', 'TS%_advanced']
  model_df[cols_to_zero] = model_df[cols_to_zero].fillna(0) # fill cols with 0
  
  cols_to_avg = ['PER_advanced', 'VORP_advanced', '3P%_totals', '2P%_totals', 'eFG%_totals']
  model_df[cols_to_avg] = model_df[cols_to_avg].fillna(model_df[cols_to_avg].mean()) # fill cols with avg
  
  cols_to_position_avg = ['TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game', 'TRB_totals', 'AST_totals', 'STL_totals', 'BLK_totals']
  model_df[cols_to_position_avg] = model_df.groupby("Position")[cols_to_position_avg].transform(lambda x: x.fillna(x.mean())) # fills cols with avg by position
  
  cols_to_fill = ['2P_per_game', '2PA_per_game', '2P_totals', '2PA_totals']
  cols_to_fill_with = ['FG_per_game', 'FGA_per_game', 'FG_totals', 'FGA_totals']
  model_df[cols_to_fill] = model_df[cols_to_fill].fillna(model_df[cols_to_fill_with]) # fill 2P shooting columns with FG columns

  return model_df
fillNulls(model_df)

,Player,Eligible,Position,Hall_of_Fame,MVP,Finals_MVP,NBA_Champ,All_NBA,All_Defensive,Def_POY,...,OWS_advanced,DWS_advanced,WS_advanced,WS/48_advanced,OBPM_advanced,DBPM_advanced,BPM_advanced,VORP_advanced,All_League,Champ
0,Alaa Abdelnaby,1,Forward,0,0,0,0,0,0,0,...,0.7,4.1,4.8,0.072,-2.9,-0.9,-3.8,-1.500000,0,0
1,Zaid Abdul-Aziz,1,Center,0,0,0,0,0,0,0,...,5.9,11.6,17.5,0.076,0.6,-0.2,0.4,2.700000,0,0
2,Kareem Abdul-Jabbar,1,Center,1,6,2,6,15,11,0,...,178.9,94.5,273.4,0.228,4.1,1.6,5.7,85.700000,15,6
3,Mahmoud Abdul-Rauf,1,Guard,0,0,0,0,0,0,0,...,16.7,8.4,25.2,0.077,0.7,-1.5,-0.8,4.500000,0,0
4,Tariq Abdul-Wahad,1,Guard,0,0,0,0,0,0,0,...,-0.6,4.1,3.5,0.035,-2.6,-0.4,-3.0,-1.200000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,Jim Zoet,1,Center,0,0,0,0,0,0,0,...,-0.1,0.0,-0.1,-0.123,-5.6,0.2,-5.4,-0.100000,0,0
4973,Bill Zopf,1,Guard,0,0,0,0,0,0,0,...,-0.5,0.4,-0.1,-0.011,NaN,NaN,0.0,3.434634,0,0
4974,Ivica Zubac,0,Center,0,0,0,0,0,0,0,...,16.4,9.6,26.1,0.183,0.3,0.4,0.6,4.500000,0,0
4975,Matt Zunic,1,Guard,0,0,0,0,0,0,0,...,0.2,1.8,2.0,NaN,NaN,NaN,0.0,3.434634,0,0


In [72]:
model_df.columns

Index(['Player', 'Eligible', 'Position', 'Hall_of_Fame', 'MVP', 'Finals_MVP',
       'NBA_Champ', 'All_NBA', 'All_Defensive', 'Def_POY', 'All_Star',
       'Scoring_Champ', 'TRB_Champ', 'AST_Champ', 'STL_Champ', 'BLK_Champ',
       'All_ABA', 'ABA_Champ', 'ROY', 'FG_per_game', 'FGA_per_game',
       '3P_per_game', '3PA_per_game', '2P_per_game', '2PA_per_game',
       'FT_per_game', 'FTA_per_game', 'ORB_per_game', 'DRB_per_game',
       'TRB_per_game', 'AST_per_game', 'STL_per_game', 'BLK_per_game',
       'PTS_per_game', 'G_totals', 'GS_totals', 'FG_totals', 'FGA_totals',
       'FG%_totals', '3P_totals', '3PA_totals', '3P%_totals', '2P_totals',
       '2PA_totals', '2P%_totals', 'eFG%_totals', 'FT_totals', 'FTA_totals',
       'FT%_totals', 'ORB_totals', 'DRB_totals', 'TRB_totals', 'AST_totals',
       'STL_totals', 'BLK_totals', 'PTS_totals', 'Trp_Dbl_totals',
       'PER_advanced', 'TS%_advanced', 'OWS_advanced', 'DWS_advanced',
       'WS_advanced', 'WS/48_advanced', 'OBPM_advanced

In [80]:
model_cols = [
    'Eligible',
    'Hall_of_Fame', 'MVP', 'Finals_MVP', 'All_Defensive', 'Def_POY', 'All_Star', 'All_League', 'Champ', 'Scoring_Champ', 'AST_Champ', # accolades
    'PER_advanced', 'VORP_advanced', 'WS_advanced', 'BPM_advanced', 'TS%_advanced', 'TRB_totals', 'AST_totals', 'STL_totals', 'BLK_totals'] # stats
df = model_df[model_cols]
df

,Eligible,Hall_of_Fame,MVP,Finals_MVP,All_Defensive,Def_POY,All_Star,All_League,Champ,Scoring_Champ,AST_Champ,PER_advanced,VORP_advanced,WS_advanced,BPM_advanced,TS%_advanced,TRB_totals,AST_totals,STL_totals,BLK_totals
0,1,0,0,0,0,0,0,0,0,0,0,13.000000,-1.500000,4.8,-3.8,0.532,846.000000,85,71.000000,69.000000
1,1,0,0,0,0,0,0,0,0,0,0,15.100000,2.700000,17.5,0.4,0.479,4065.000000,601,131.000000,205.000000
2,1,1,6,2,11,0,19,15,6,2,0,24.600000,85.700000,273.4,5.7,0.592,17440.000000,5660,1160.000000,3189.000000
3,1,0,0,0,0,0,0,0,0,0,0,15.400000,4.500000,25.2,-0.8,0.506,1087.000000,2079,487.000000,46.000000
4,1,0,0,0,0,0,0,0,0,0,0,11.400000,-1.200000,3.5,-3.0,0.467,776.000000,266,184.000000,83.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4972,1,0,0,0,0,0,0,0,0,0,0,-0.800000,-0.100000,-0.1,-5.4,0.200,8.000000,1,1.000000,3.000000
4973,1,0,0,0,0,0,0,0,0,0,0,9.600000,3.434634,-0.1,0.0,0.391,46.000000,73,269.086642,54.094918
4974,0,0,0,0,0,0,0,0,0,0,0,19.200000,4.500000,26.1,0.6,0.636,2350.000000,410,112.000000,304.000000
4975,1,0,0,0,0,0,0,0,0,0,0,11.212562,3.434634,2.0,0.0,0.368,68.357143,50,269.086642,54.094918


In [84]:
eligible_df = df[df['Eligible'] == 1]
noneligible_df = df[df['Eligible'] == 0]

In [86]:
X_eligible = eligible_df.iloc[:, 2:].values
y_eligible = eligible_df.iloc[:, 1].values

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.50000000e+01, 7.10000000e+01, 6.90000000e+01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.01000000e+02, 1.31000000e+02, 2.05000000e+02],
       [6.00000000e+00, 2.00000000e+00, 1.10000000e+01, ...,
        5.66000000e+03, 1.16000000e+03, 3.18900000e+03],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.30000000e+01, 2.69086642e+02, 5.40949177e+01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.00000000e+01, 2.69086642e+02, 5.40949177e+01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.44000000e+02, 2.69086642e+02, 5.40949177e+01]])

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_eligible, y_eligible, test_size = 0.25, random_state = 0)

In [ ]:
def standardizeCols(model_df, numeric_cols):
  scaler = StandardScaler()
  model_df[numeric_cols] = scaler.fit_transform(model_df[numeric_cols])
  return model_df